In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

#### Extend stopwords

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'room', 'apartment', 'bedroom', 'home', 'bed', 'bathroom', 'kitchen',
                  'NEIGHBORHOOD', 'neighborhood'])

#### Load dataset

In [7]:
columns = ['ID', 'Name', 'Summary', 'Space', 'Description', 'Neighborhood Overview', 'House Rules']
df = pd.read_csv("../../data/airbnb-data-science/airbnb-listings.csv", usecols=columns, sep=';')
df = df[columns]
#df = pd.read_csv("../../data/airbnb-data-science/airbnb-listings-random100nyc.csv")
#df.head(2)

/Users/nesara/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Extract listings description

In [35]:
# Convert to list
data = df.sample(n=10000, random_state=1)["Description"].values.tolist()

#### Tokenise words

In [37]:
def sent_to_words(sentences):
    i = 0
    for sentence in sentences:
        #if i%10000 == 0:
        #    print(i)
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        i+=1

data_words = list(sent_to_words(data))

#### Create bigram and trigram models

In [38]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

In [39]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#### Create the Dictionary and Corpus needed for Topic Modeling

In [40]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

#### Building the Topic Model

In [41]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

#### View the topics in LDA model 

In [42]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"floor" + 0.025*"large" + 0.019*"double" + 0.019*"area" + '
  '0.015*"live" + 0.014*"modern" + 0.013*"living" + 0.013*"bedroom" + '
  '0.012*"spacious" + 0.012*"locate"'),
 (1,
  '0.032*"deck" + 0.024*"beach" + 0.017*"und" + 0.017*"die" + 0.013*"standard" '
  '+ 0.012*"villa" + 0.011*"yard" + 0.010*"be" + 0.009*"cottage" + '
  '0.009*"mountain"'),
 (2,
  '0.035*"private" + 0.027*"guest" + 0.022*"full" + 0.018*"access" + '
  '0.017*"parking" + 0.016*"include" + 0.015*"downtown" + 0.015*"provide" + '
  '0.014*"space" + 0.014*"free"'),
 (3,
  '0.024*"walk" + 0.023*"minute" + 0.019*"place" + 0.018*"restaurant" + '
  '0.016*"close" + 0.013*"good" + 0.012*"away" + 0.012*"great" + '
  '0.012*"locate" + 0.011*"station"'),
 (4,
  '0.025*"appartement" + 0.024*"dan" + 0.024*"pour" + 0.017*"vous" + '
  '0.013*"tre" + 0.012*"pie" + 0.011*"due" + 0.011*"acce" + 0.010*"immeuble" + '
  '0.009*"et"')]


#### Compute Model Perplexity and Coherence Score

In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.230763477366619

Coherence Score:  0.5528674075941595


#### Visualize the topics-keywords

In [44]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/nesara/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.191983  0.145919       1        1  45.568829
0     -0.193646  0.117527       2        1  28.340424
2     -0.161277 -0.288853       3        1  21.166157
4      0.258981  0.108086       4        1   2.510139
1      0.287924 -0.082678       5        1   2.414453, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
121   Default  3465.000000        private  3465.000000  30.0000  30.0000
29    Default  4742.000000         minute  4742.000000  29.0000  29.0000
193   Default  3179.000000          floor  3179.000000  28.0000  28.0000
79    Default  3146.000000          large  3146.000000  27.0000  27.0000
116   Default  4793.000000           walk  4793.000000  26.0000  26.0000
19    Default  3014.000000          guest  3014.000000  25.0000  25.0000
197   Default  2087.000000           full  2087.000000  24.0000  24.0000
92    Default  3952.000000          place  3952.000000  23.0000  23.0000
134   Default  2446.000000         double  2446.000000  22.0000  22.0000
54    Default  3703.000000     restaurant  3703.000000  21.0000  21.0000
50    Default  3315.000000          close  3315.000000  20.0000  20.0000
278   Default  1590.000000        parking  1590.000000  19.0000  19.0000
126   Default   715.000000          beach   715.000000  18.0000  18.0000
298   Default  1424.000000       downtown  1424.000000  17.0000  17.0000
234   Default  1756.000000         modern  1756.000000  16.0000  16.0000
250   Default  2903.000000         access  2903.000000  15.0000  15.0000
51    Default  2624.000000           good  2624.000000  14.0000  14.0000
3     Default  1604.000000        bedroom  1604.000000  13.0000  13.0000
411   Default  1507.000000        provide  1507.000000  12.0000  12.0000
78    Default  2000.000000        include  2000.000000  11.0000  11.0000
49    Default  1266.000000          block  1266.000000  10.0000  10.0000
57    Default  2519.000000           away  2519.000000   9.0000   9.0000
47    Default  2171.000000            bar  2171.000000   8.0000   8.0000
55    Default  4320.000000           area  4320.000000   7.0000   7.0000
176   Default  1226.000000           size  1226.000000   6.0000   6.0000
16    Default  1550.000000           free  1550.000000   5.0000   5.0000
428   Default   984.000000         couple   984.000000   4.0000   4.0000
449   Default  1848.000000         living  1848.000000   3.0000   3.0000
36    Default  2297.000000        station  2297.000000   2.0000   2.0000
56    Default  2267.000000      available  2267.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1558   Topic5   182.289810            die   183.030777   3.7196  -4.0700
875    Topic5   143.960022       standard   144.735626   3.7183  -4.3060
458    Topic5   122.910202          villa   123.658096   3.7176  -4.4641
2097   Topic5   115.278725           yard   116.032906   3.7172  -4.5282
1194   Topic5   111.845360             be   112.590164   3.7171  -4.5584
2092   Topic5    99.259315        cottage   100.009064   3.7162  -4.6778
84     Topic5    97.483597       mountain    98.235046   3.7160  -4.6959
2842   Topic5    83.437683  swimming_pool    84.205925   3.7145  -4.8515
1476   Topic5    81.367729            non    82.148346   3.7141  -4.8766
1472   Topic5    77.593056             ca    78.354607   3.7139  -4.9241
1562   Topic5    68.181541        organic    68.944969   3.7126  -5.0534
1228   Topic5    64.702888        weather    65.476166   3.7118  -5.1058
850    Topic5    63.685673         island    64.455437   3.7117  -5.1216
1123   Topic5    63.822422             do    64.623131   3.7112  -5.1195
829    Topic5    59.977077    spectacular    60.734921   3.7111  -5.1816
870    Topic5    60.944031        mention    61.725151   3.7110  -5.1656
21